In [1]:
from utility_common import *
from utility_plotter import *
#import ROOT
from IPython.display import clear_output
## add plotting option and load plotting parameter table
%matplotlib inline
#%qtconsole --style monokai
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

In [2]:
ctr = SelectionCounter()

nGenTT = np.array([1811409.0, 1811532.0, 3620281.0, 57057.0, 54227.0, 111125.0, 418126.0, 407605.0, 763147.0, 642646.0, 626108.0, 2350887.0, 643368.0, 627277.0, 2353727.0, 22670017.0, 22653517.0, 4016590.0, 3923504.0, 14718933.0, 70930278.0])
nGenT  = np.array([23105.0, 23040.0, 46342.0, 694.0, 717.0, 1420.0, 5260.0, 5158.0, 9727.0, 8199.0, 8029.0, 30054.0, 8163.0, 8006.0, 29970.0, 290101.0, 289467.0, 50864.0, 50050.0, 187515.0, 904067.0])

        
# print Generator information
# f = ROOT.TFile("/mnt/data/zchen/Analysis/root/2016DataMC.root")
# nGenTT = np.array(ctr.countGen_ByTauDecay(f,'ttbar_inclusive'))

# nGenT = np.array(ctr.countGen_ByTauDecay(f,'t_tw'))
# nGenT += np.array(ctr.countGen_ByTauDecay(f,'tbar_tw'))
# f.Close()

def getPickleDir(selection):
    if selection == "emu2":
        pickledir  =  "/mnt/data/zchen/Analysis/pickle/emu/"
    else:
        pickledir  =  "/mnt/data/zchen/Analysis/pickle/{}/".format(selection)
    return pickledir
    

# Get Acc

In [15]:
dirAcc = "/mnt/data/zchen/Analysis/acceptance/"
for selection in ["mumu","mutau","mu4j","ee","etau","e4j","emu","emu2"]:
    print(selection)
    result = []
    
    pickledir  =  getPickleDir(selection)

    for mc in ["mctt","mct"]:
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection, nbjetcut)
            
            MC = LoadDataframe(pickledir + mc).query(cuts)
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC,withWeights=True))
            if mc == "mctt":
                fractionAccepted = nAcc/nGenTT
            if mc == "mct":
                fractionAccepted = nAcc/nGenT
            result.append(fractionAccepted)
    result = np.array(result)
    np.savetxt(dirAcc+'Acceptance - {}.csv'.format(selection), result, delimiter=',')

mumu
mutau
mu4j
ee
etau
e4j
emu
emu2


# ACC with Jet corrections

In [16]:
dirAcc = "/mnt/data/zchen/Analysis/acceptance/"
for selection in ["mumu","mutau","mu4j","ee","etau","e4j","emu","emu2"]:
    print(selection)
    MCtt = LoadDataframe(getPickleDir(selection)+"mctt")
    MCt  = LoadDataframe(getPickleDir(selection)+"mct")
    for name in ["JESUp","JESDown","JERUp","JERDown","BTagUp","BTagDown","MistagUp","MistagDown"]:    
        
        result = []

        
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection, nbjetcut, shiftJet=name)
            MC   = MCtt.query(cuts)
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC))
            result.append(nAcc/nGenTT)
        
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection, nbjetcut, shiftJet=name)
            MC   = MCt.query(cuts)
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC))
            result.append(nAcc/nGenT)
            
        result = np.array(result)
        np.savetxt(dirAcc+'Acc_{}_{}.csv'.format(name,selection), result, delimiter=',')

mumu
mutau
mu4j
ee
etau
e4j
emu
emu2


# ACC with tt PDF and QCD variation

In [13]:
variables = {"RenormUp":"qcd_weight_up_nom",
             "RenormDown":"qcd_weight_down_nom",
             "FactorUp":"qcd_weight_nom_up",
             "FactorDown":"qcd_weight_nom_down",
             "PDFUp":"pdf_weight_up",
             "PDFDown":"pdf_weight_down"
            }

In [14]:
dirAcc = "/mnt/data/zchen/Analysis/acceptance/"

for selection in ["mumu","mutau","mu4j","ee","etau","e4j","emu","emu2"]:
    print(selection)
    MCtt = LoadDataframe(getPickleDir(selection)+"mctt")
    MCt  = LoadDataframe(getPickleDir(selection)+"mct")
    
    MCtt["pdf_weight_up"] = 1+np.sqrt(MCtt.pdf_var/99)
    MCtt["pdf_weight_down"] = 1-np.sqrt(MCtt.pdf_var/99)
    
    for name in ["RenormUp","RenormDown","FactorUp","FactorDown","PDFUp","PDFDown"]:
    
        result = []
    
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection, nbjetcut)
            MC   = MCtt.query(cuts)
            
            variable = variables[name]
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC, withWeightsAdjust = MC[variable] ))
            result.append(nAcc/nGenTT)
        
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection, nbjetcut)
            MC   = MCt.query(cuts)
            
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC))
            result.append(nAcc/nGenT)
                
        result = np.array(result)
        np.savetxt(dirAcc+'Acc_{}_{}.csv'.format(name,selection), result, delimiter=',')

mumu
mutau
mu4j
ee
etau
e4j
emu
emu2


# tt Theoretical Uncertainty

In [3]:
ctr = SelectionCounter()
# print Generator information
f = ROOT.TFile("/mnt/data/zchen/Analysis/root/2016DataMC.root")
nGenT = np.array(ctr.countGen_ByTauDecay(f,'t_tw'))
nGenT += np.array(ctr.countGen_ByTauDecay(f,'tbar_tw'))
f.Close()

In [5]:
for name in ["up","down","fsrup","fsrdown","isrup","isrdown","hdampup","hdampdown"]:
    f = ROOT.TFile("/mnt/data/zchen/Analysis/root/2016DataMC_ttTheoreticUnc.root")
    nGenTT = np.array(ctr.countGen_ByTauDecay(f,'ttbar_inclusive_{}'.format(name)))
    f.Close()

    dirAcc = "/mnt/data/zchen/Analysis/acceptance/"
    for selection in ["mumu","mutau","mu4j","ee","etau","e4j","emu","emu2"]:
        print(selection)
        result = []

        if selection == "emu2":
            pickledir  =  "/mnt/data/zchen/Analysis/pickle/emu/"
        else:
            pickledir  =  "/mnt/data/zchen/Analysis/pickle/{}/".format(selection)
        
        mc = pickledir + "mcttTheory/ntuple_ttbar_inclusive_"+name+".pkl"
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection) + "& (nBJets{})".format(nbjetcut)
            MC   = pd.read_pickle(mc).query(cuts)
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC,withweights=True))
            fractionAccepted = nAcc/nGenTT
            result.append(fractionAccepted)
        
        mc = pickledir + "mct"
        for nbjetcut in ["==1",">1"]:
            cuts = GetSelectionCut(selection) + "& (nBJets{})".format(nbjetcut)
            MC = LoadDataframe(mc).query(cuts)
            nAcc = np.array(ctr.countSlt_ByTauDecay(MC,withweights=True))
            
            fractionAccepted = nAcc/nGenT
            result.append(fractionAccepted)
        
        result = np.array(result)
            
        np.savetxt(dirAcc+'Acc_{}_{}.csv'.format(name,selection), result, delimiter=',')

NameError: name 'ROOT' is not defined